![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Financial Deidentification

## Setup

In [1]:
from johnsnowlabs import *

import json
import os

spark = start_spark()

📋 Loading license number 0 from /home/david/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json


your 131072x1 screen size is bogus. expect trouble
24/11/11 08:13:45 WARN Utils: Your hostname, DadaBook resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/11 08:13:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/11/11 08:13:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.4.0, 💊Spark-Healthcare==5.4.0, running on ⚡ PySpark==3.4.0


# Deidentification Model

Some financial information can be considered sensitive. (e.g.,document, organization, address, signer)

In [2]:
documentAssembler = nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")

sentenceDetector = (
    nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")
    .setInputCols(["document"])
    .setOutputCol("sentence")
)
# .setCustomBounds(["\n\n"])

tokenizer = nlp.Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

embeddings = (
    nlp.WordEmbeddingsModel.pretrained(
        "finance_word_embeddings", "en", "finance/models"
    )
    .setInputCols(["sentence", "token"])
    .setOutputCol("embeddings")
)

fin_ner = (
    finance.NerModel.pretrained("finner_deid_sec_fe", "en", "finance/models")
    .setInputCols(["sentence", "token", "embeddings"])
    .setOutputCol("ner")
)

ner_converter = (
    finance.NerConverter()
    .setInputCols(["sentence", "token", "ner"])
    .setOutputCol("ner_chunk")
    .setReplaceLabels({"ORG": "PARTY"})
)  # Replace "ORG" entity as "PARTY"

nlp_pipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        fin_ner,
        ner_converter,
    ]
)

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlp_pipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ]

24/11/11 08:14:52 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/11/11 08:14:52 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
Download done! Loading the resource.
[ / ]

2024-11-11 08:14:56.538797: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]
finance_word_embeddings download started this may take some time.
Approximate size to download 98.6 MB
[ | ]

...

finance_word_embeddings download started this may take some time.
Approximate size to download 98.6 MB
[ / ]Download done! Loading the resource.
[OK!]
finner_deid_sec_fe download started this may take some time.
[ | ]

24/11/11 08:15:13 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/11/11 08:15:13 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


finner_deid_sec_fe download started this may take some time.
Approximate size to download 13.9 MB
[ / ]Download done! Loading the resource.
[ — ]

2024-11-11 08:15:38.201530: E external/org_tensorflow/tensorflow/core/framework/node_def_util.cc:630] NodeDef mentions attribute validate_shape which is not in the op definition: Op<name=AssignVariableOp; signature=resource:resource, value:dtype -> ; attr=dtype:type; is_stateful=true> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node char_repr_cnn/conv1d/kernel/Assign}}
2024-11-11 08:15:38.577802: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member

[OK!]


### Pretrained NER models extracts:
- PROFESSION
- URL
- LOCATION-OTHER
- CITY
- DATE
- ZIP
- PERSON
- STATE
- COUNTRY
- STREET
- ORG
- PHONE
- EMAIL
- FAX
- AGE

In [3]:
fin_ner.getClasses()

['O',
 'I-PROFESSION',
 'B-PROFESSION',
 'B-URL',
 'I-LOCATION-OTHER',
 'I-URL',
 'B-CITY',
 'B-DATE',
 'I-ZIP',
 'I-PERSON',
 'B-LOCATION-OTHER',
 'B-STATE',
 'I-STATE',
 'B-PERSON',
 'I-CITY',
 'I-DATE',
 'B-COUNTRY',
 'B-ZIP',
 'I-STREET',
 'B-ORG',
 'I-ORG',
 'B-PHONE',
 'I-PHONE',
 'B-EMAIL',
 'B-STREET',
 'B-FAX',
 'B-AGE',
 'I-FAX',
 'I-AGE',
 'I-COUNTRY']

In [4]:
text = """
(State or other jurisdictionof incorporation or organization)
(I.R.S. EmployerIdentification No.)
55 Almaden Boulevard, 6th Floor
San Jose, California 95113
(Address of principal executive offices and Zip Code)
799-9666
(Registrant’s telephone number, including area code) """

In [5]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [6]:
import pyspark.sql.functions as F

result_df = result.select(
    F.explode(F.arrays_zip(result.token.result, result.ner.result)).alias("cols")
).select(F.expr("cols['0']").alias("token"), F.expr("cols['1']").alias("ner_label"))

In [7]:
result_df.select("token", "ner_label").groupBy("ner_label").count().orderBy(
    "count", ascending=False
).show(truncate=False)

+------------+-----+
|ner_label   |count|
+------------+-----+
|O           |36   |
|I-STREET    |2    |
|I-CITY      |1    |
|B-PHONE     |1    |
|B-STATE     |1    |
|I-PROFESSION|1    |
|B-STREET    |1    |
|I-ZIP       |1    |
|B-CITY      |1    |
|B-PROFESSION|1    |
+------------+-----+



### Check extracted sensitive entities

In [8]:
result.select(
    F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata)).alias(
        "cols"
    )
).select(
    F.expr("cols['0']").alias("chunk"), F.expr("cols['1']['entity']").alias("ner_label")
).show(truncate=False)

+--------------------+----------+
|chunk               |ner_label |
+--------------------+----------+
|55 Almaden Boulevard|STREET    |
|San Jose            |CITY      |
|California          |STATE     |
|95113               |ZIP       |
|principal executive |PROFESSION|
|799-9666            |PHONE     |
+--------------------+----------+



## Masking and Obfuscation

### Replace these enitites with Tags

In [9]:
ner_converter = (
    finance.NerConverter()
    .setInputCols(["sentence", "token", "ner"])
    .setOutputCol("ner_chunk")
)

deidentification = (
    finance.DeIdentification()
    .setInputCols(["sentence", "token", "ner_chunk"])
    .setOutputCol("deidentified")
    .setMode("mask")
    .setReturnEntityMappings(True)
)  #  return a new column to save the mappings between the mask/obfuscated entities and original entities.
# .setMappingsColumn("MappingCol") # change the name of the column, 'aux' is default

deidPipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        fin_ner,
        ner_converter,
        deidentification,
    ]
)

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_deid = deidPipeline.fit(empty_data)

In [10]:
result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [11]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|        deidentified|                 aux|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|\n(State or other...|[{document, 0, 27...|[{document, 1, 97...|[{token, 1, 1, (,...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 99, 118,...|[{document, 0, 96...|[{chunk, 97, 104,...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [12]:
result.select(
    F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias(
        "cols"
    )
).select(
    F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")
).toPandas()

,sentence,deidentified
0,(State or other jurisdictionof incorporation o...,(State or other jurisdictionof incorporation o...
1,"55 Almaden Boulevard, 6th Floor","<STREET>, 6th Floor"
2,"San Jose, California 95113","<CITY>, <STATE> <ZIP>"
3,(Address of principal executive offices and Zi...,(Address of <PROFESSION> offices and Zip Code)...
4,"(Registrant’s telephone number, including area...","(Registrant’s telephone number, including area..."


We have three modes to mask the entities in the Deidentification annotator. You can select the modes using the `.setMaskingPolicy()` parameter. The methods are the followings:

**“entity_labels”**: Mask with the entity type of that chunk. (default) <br/>
**“same_length_chars”**: Mask the deid entities with same length of asterix ( * ) with brackets ( [ , ] ) on both end. <br/>
**“fixed_length_chars”**: Mask the deid entities with a fixed length of asterix ( * ). The length is setting up using the `setFixedMaskLength()` method. <br/>

Let's try each of these and compare the results:

In [13]:
# deid model with "entity_labels"
deid_entity_labels = (
    finance.DeIdentification()
    .setInputCols(["sentence", "token", "ner_chunk"])
    .setOutputCol("deid_entity_label")
    .setMode("mask")
    .setReturnEntityMappings(True)
    .setMaskingPolicy("entity_labels")
)

# deid model with "same_length_chars"
deid_same_length = (
    finance.DeIdentification()
    .setInputCols(["sentence", "token", "ner_chunk"])
    .setOutputCol("deid_same_length")
    .setMode("mask")
    .setReturnEntityMappings(True)
    .setMaskingPolicy("same_length_chars")
)

# deid model with "fixed_length_chars"
deid_fixed_length = (
    finance.DeIdentification()
    .setInputCols(["sentence", "token", "ner_chunk"])
    .setOutputCol("deid_fixed_length")
    .setMode("mask")
    .setReturnEntityMappings(True)
    .setMaskingPolicy("fixed_length_chars")
    .setFixedMaskLength(4)
)


deidPipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        fin_ner,
        ner_converter,
        deid_entity_labels,
        deid_same_length,
        deid_fixed_length,
    ]
)


empty_data = spark.createDataFrame([[""]]).toDF("text")

model_deid = deidPipeline.fit(empty_data)

In [14]:
policy_result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [15]:
policy_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|   deid_entity_label|                 aux|    deid_same_length|   deid_fixed_length|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|\n(State or other...|[{document, 0, 27...|[{document, 1, 97...|[{token, 1, 1, (,...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 99, 118,...|[{document, 0, 96...|[{chunk, 97, 100,...|[{document, 0, 96...|[{document, 0, 96...|
+--------------------+--------------------+--------------------+----

In [16]:
policy_result.select(
    F.explode(
        F.arrays_zip(
            policy_result.sentence.result,
            policy_result.deid_entity_label.result,
            policy_result.deid_same_length.result,
            policy_result.deid_fixed_length.result,
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("sentence"),
    F.expr("cols['1']").alias("deid_entity_label"),
    F.expr("cols['2']").alias("deid_same_length"),
    F.expr("cols['3']").alias("deid_fixed_length"),
).toPandas()

,sentence,deid_entity_label,deid_same_length,deid_fixed_length
0,(State or other jurisdictionof incorporation o...,(State or other jurisdictionof incorporation o...,(State or other jurisdictionof incorporation o...,(State or other jurisdictionof incorporation o...
1,"55 Almaden Boulevard, 6th Floor","<STREET>, 6th Floor","[******************], 6th Floor","****, 6th Floor"
2,"San Jose, California 95113","<CITY>, <STATE> <ZIP>","[******], [********] [***]","****, **** ****"
3,(Address of principal executive offices and Zi...,(Address of <PROFESSION> offices and Zip Code)...,(Address of [*****************] offices and Zi...,(Address of **** offices and Zip Code)\n****
4,"(Registrant’s telephone number, including area...","(Registrant’s telephone number, including area...","(Registrant’s telephone number, including area...","(Registrant’s telephone number, including area..."


### Mapping Column

In [17]:
result.select("aux").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
result.select(
    F.explode(
        F.arrays_zip(
            result.aux.metadata, result.aux.result, result.aux.begin, result.aux.end
        )
    ).alias("cols")
).select(
    F.expr("cols['0']['originalChunk']").alias("chunk"),
    F.expr("cols['0']['beginOriginalChunk']").alias("beginChunk"),
    F.expr("cols['0']['endOriginalChunk']").alias("endChunk"),
    F.expr("cols['1']").alias("label"),
    F.expr("cols['2']").alias("beginLabel"),
    F.expr("cols['3']").alias("endLabel"),
).show(truncate=False)

+--------------------+----------+--------+------------+----------+--------+
|chunk               |beginChunk|endChunk|label       |beginLabel|endLabel|
+--------------------+----------+--------+------------+----------+--------+
|55 Almaden Boulevard|99        |118     |<STREET>    |97        |104     |
|San Jose            |131       |138     |<CITY>      |116       |121     |
|California          |141       |150     |<STATE>     |124       |130     |
|95113               |152       |156     |<ZIP>       |132       |136     |
|principal executive |170       |188     |<PROFESSION>|149       |160     |
|799-9666            |212       |219     |<PHONE>     |184       |190     |
+--------------------+----------+--------+------------+----------+--------+



## Reidentification

We can use `ReIdentification` annotator to go back to the original sentence.

In [19]:
reIdentification = (
    finance.ReIdentification()
    .setInputCols(["aux", "deidentified"])
    .setOutputCol("original")
)

In [20]:
reid_result = reIdentification.transform(result)

In [21]:
reid_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|        deidentified|                 aux|            original|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|\n(State or other...|[{document, 0, 27...|[{document, 1, 97...|[{token, 1, 1, (,...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 99, 118,...|[{document, 0, 96...|[{chunk, 97, 104,...|[{document, 1, 97...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----

In [22]:
print(text)

reid_result.select("original.result").show(truncate=False)


(State or other jurisdictionof incorporation or organization)
(I.R.S. EmployerIdentification No.)
55 Almaden Boulevard, 6th Floor
San Jose, California 95113
(Address of principal executive offices and Zip Code)
799-9666
(Registrant’s telephone number, including area code) 


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[(State or other jurisdictionof incorporation or organization)\n(I.R.S. EmployerIdentification No.), 55 Almaden Boulevard, 6th Floor, San Jose, Califo

## Using multiple NER in the same pipeline

In [23]:
documentAssembler = nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")

sentenceDetector = (
    nlp.SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")
)

tokenizer = nlp.Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

embeddings = (
    nlp.WordEmbeddingsModel.pretrained(
        "finance_word_embeddings", "en", "finance/models"
    )
    .setInputCols(["sentence", "token"])
    .setOutputCol("embeddings")
)

bert_embeddings = (
    nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base", "en")
    .setInputCols(["sentence", "token"])
    .setOutputCol("bert_embeddings")
)

fin_ner = (
    finance.NerModel.pretrained("finner_deid_sec_fe", "en", "finance/models")
    .setInputCols(["sentence", "token", "embeddings"])
    .setOutputCol("ner")
)
# .setLabelCasing("upper")

ner_converter = (
    finance.NerConverter()
    .setInputCols(["sentence", "token", "ner"])
    .setOutputCol("ner_chunk")
    .setReplaceLabels({"ORG": "PARTY"})
)  # Replace "ORG" entity as "PARTY"

ner_finner = (
    finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")
    .setInputCols(["sentence", "token", "bert_embeddings"])
    .setOutputCol("ner_finner")
)
# .setLabelCasing("upper")

ner_converter_finner = (
    nlp.NerConverter()
    .setInputCols(["sentence", "token", "ner_finner"])
    .setOutputCol("ner_finner_chunk")
    .setWhiteList(["ROLE"])
)  # Just use "ROLE" entity from this NER

chunk_merge = (
    finance.ChunkMergeApproach()
    .setInputCols("ner_finner_chunk", "ner_chunk")
    .setOutputCol("deid_merged_chunk")
)

deidentification = (
    finance.DeIdentification()
    .setInputCols(["sentence", "token", "deid_merged_chunk"])
    .setOutputCol("deidentified")
    .setMode("mask")
    .setIgnoreRegex(True)
)


nlp_pipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        bert_embeddings,
        fin_ner,
        ner_converter,
        ner_finner,
        ner_converter_finner,
        chunk_merge,
        deidentification,
    ]
)

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlp_pipeline.fit(empty_data)

finance_word_embeddings download started this may take some time.


24/11/11 08:22:48 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 98.6 MB
[OK!]
bert_embeddings_sec_bert_base download started this may take some time.


24/11/11 08:22:52 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 387.7 MB
[ | ]

...

finner_org_per_role_date download started this may take some time.
Approximate size to download 15.4 MB
[ / ]Download done! Loading the resource.


2024-11-11 08:23:44.809685: E external/org_tensorflow/tensorflow/core/framework/node_def_util.cc:630] NodeDef mentions attribute validate_shape which is not in the op definition: Op<name=AssignVariableOp; signature=resource:resource, value:dtype -> ; attr=dtype:type; is_stateful=true> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node char_repr_cnn/conv1d/kernel/Assign}}


[ — ]

2024-11-11 08:23:45.108469: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
AddV2: CPU 
Unique: CPU 
Mul: CPU 
VariableV2: CPU 
Assign: CPU 
Shape: CPU 
Identity: CPU 
Cast: CPU 
Sub: CPU 
GatherV2: CPU

[OK!]


In [24]:
text = """ Jeffrey Preston Bezos is an American entrepreneur, founder and CEO of Amazon  """

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [25]:
# fin_ner
result.select(
    F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata)).alias(
        "cols"
    )
).select(
    F.expr("cols['0']").alias("chunk"), F.expr("cols['1']['entity']").alias("ner_label")
).show(truncate=False)

+---------------------+---------+
|chunk                |ner_label|
+---------------------+---------+
|Jeffrey Preston Bezos|PERSON   |
|Amazon               |PARTY    |
+---------------------+---------+



In [26]:
# ner_finner
result.select(
    F.explode(
        F.arrays_zip(result.ner_finner_chunk.result, result.ner_finner_chunk.metadata)
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("chunk"), F.expr("cols['1']['entity']").alias("ner_label")
).show(truncate=False)

24/11/11 08:24:51 WARN DAGScheduler: Broadcasting large task binary with size 1004.3 KiB
24/11/11 08:24:51 WARN DAGScheduler: Broadcasting large task binary with size 1004.3 KiB
24/11/11 08:24:52 WARN DAGScheduler: Broadcasting large task binary with size 1004.3 KiB


+-------+---------+
|chunk  |ner_label|
+-------+---------+
|founder|ROLE     |
|CEO    |ROLE     |
+-------+---------+



In [27]:
# merged_chunk
result.select(
    F.explode(
        F.arrays_zip(result.deid_merged_chunk.result, result.deid_merged_chunk.metadata)
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("chunk"), F.expr("cols['1']['entity']").alias("ner_label")
).show(truncate=False)

24/11/11 08:25:00 WARN DAGScheduler: Broadcasting large task binary with size 1058.9 KiB
24/11/11 08:25:00 WARN DAGScheduler: Broadcasting large task binary with size 1058.9 KiB
24/11/11 08:25:00 WARN DAGScheduler: Broadcasting large task binary with size 1058.9 KiB


+---------------------+---------+
|chunk                |ner_label|
+---------------------+---------+
|Jeffrey Preston Bezos|PERSON   |
|founder              |ROLE     |
|CEO                  |ROLE     |
|Amazon               |PARTY    |
+---------------------+---------+



In [28]:
result.select(
    F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias(
        "cols"
    )
).select(
    F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")
).toPandas()

24/11/11 08:25:06 WARN DAGScheduler: Broadcasting large task binary with size 1099.4 KiB


,sentence,deidentified
0,Jeffrey Preston Bezos is an American entrepren...,"<PERSON> is an American entrepreneur, <ROLE> a..."


## Obfuscation mode

In the obfuscation mode **DeIdentificationModel** will replace sensitive entities with random values of the same type. 


In [29]:
# This is the obfuscation dict for the new entities
obs_lines = """5417543010#PHONE
(123)123-1234#PHONE
+18087339090#PHONE
(555) 555-1234#PHONE
541-700-3010#PHONE
HenryWatson@world.com#EMAIL
yousef@jacob.com#EMAIL
eric.shannon@geegle.com#EMAIL
mgt@jsl.com#EMAIL
gokhan@company.com#EMAIL
richard@company.it#EMAIL
TURER INC#PARTY
Clarus llc.#PARTY
SESA CO.#PARTY
John Snow Labs Inc#PARTY
MGT Trust Company, LLC.#PARTY
26-06-1990#EFFDATE
03/08/2025#EFFDATE
01/01/2045#EFFDATE
11/7/2016#EFFDATE
12-12-2022#EFFDATE
CEO#ROLE
CTO#ROLE
Director#ROLE
James Turner#PERSON
JUAN RAMIREZ#PERSON
Benjamin Curie#PERSON"""

with open("obfuscate.txt", "w") as f:
    f.write(obs_lines)

In [30]:
ner_converter_finner = (
    nlp.NerConverter()
    .setInputCols(["sentence", "token", "ner_finner"])
    .setOutputCol("ner_finner_chunk")
    .setWhiteList(["ROLE"])
)
chunk_merge = (
    finance.ChunkMergeApproach()
    .setInputCols("ner_finner_chunk", "ner_chunk")
    .setOutputCol("deid_merged_chunk")
)

obfuscation = (
    finance.DeIdentification()
    .setInputCols(["sentence", "token", "deid_merged_chunk"])
    .setOutputCol("deidentified")
    .setMode("obfuscate")
    .setObfuscateDate(True)
    .setObfuscateRefFile("obfuscate.txt")
    .setObfuscateRefSource("both")
)  # default: "faker"


nlp_pipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        bert_embeddings,
        fin_ner,
        ner_converter,
        ner_finner,
        ner_converter_finner,
        chunk_merge,
        obfuscation,
    ]
)

obfuscation_model = nlp_pipeline.fit(empty_data)

In [31]:
result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [32]:
result.select(
    F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias(
        "cols"
    )
).select(
    F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")
).toPandas()

24/11/11 08:25:30 WARN DAGScheduler: Broadcasting large task binary with size 1099.4 KiB


,sentence,deidentified
0,Jeffrey Preston Bezos is an American entrepren...,"Rubie Corona is an American entrepreneur, CEO ..."


## Use full pipeline in the Light model

In [33]:
light_model = nlp.LightPipeline(model)

annotated_text = light_model.annotate(text)


In [34]:
annotated_text["deidentified"]

['<PERSON> is an American entrepreneur, <ROLE> and <ROLE> of <PARTY>']

In [35]:
obf_light_model = nlp.LightPipeline(obfuscation_model)

annotated_text = obf_light_model.annotate(text)


In [36]:
annotated_text["deidentified"]

['Rubie Corona is an American entrepreneur, CEO and CTO of Sovereign Investments, LLC']

# Save the Pipeline and Use it from Your Local

In [37]:
model.write().overwrite().save("pipeline_deid")

In [38]:
# from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = nlp.PretrainedPipeline.from_disk("pipeline_deid")

Using CPUs


2024-11-11 08:27:25.152253: E external/org_tensorflow/tensorflow/core/framework/node_def_util.cc:630] NodeDef mentions attribute validate_shape which is not in the op definition: Op<name=AssignVariableOp; signature=resource:resource, value:dtype -> ; attr=dtype:type; is_stateful=true> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node char_repr_cnn/conv1d/kernel/Assign}}
2024-11-11 08:27:25.399570: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member

In [39]:
data = spark.createDataFrame([[text]]).toDF("text")

In [40]:
deid_pipeline.model.stages

[DocumentAssembler_496baa5e0d47,
 SentenceDetector_cbe8928991a1,
 REGEX_TOKENIZER_f6e8752f96c6,
 WORD_EMBEDDINGS_MODEL_a0bd8be90a24,
 BERT_EMBEDDINGS_714b8f59d9fc,
 LegalNerModel_7979324fffcf,
 NER_CONVERTER_b38c4704c333,
 MedicalNerModel_7711a4bfd1fa,
 NerConverter_fbcdd23f2361,
 MERGE_b94324a03972,
 DE-IDENTIFICATION_7c5d91954258]

In [41]:
result = deid_pipeline.model.transform(data)

In [42]:
result.show()

24/11/11 08:27:44 WARN DAGScheduler: Broadcasting large task binary with size 1401.5 KiB
24/11/11 08:27:44 WARN DAGScheduler: Broadcasting large task binary with size 1401.5 KiB
24/11/11 08:27:45 WARN DAGScheduler: Broadcasting large task binary with size 1401.5 KiB


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|     bert_embeddings|                 ner|           ner_chunk|          ner_finner|    ner_finner_chunk|   deid_merged_chunk|        deidentified|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Jeffrey Preston ...|[{document, 0, 78...|[{document, 1, 76...|[{token, 1, 7, Je...|[{word_embeddings...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 1, 21, J...|[{named_entity, 1...|[{chunk, 52, 58, ...|[{chunk, 1, 21, J...|[{docu